In [1]:
!pip install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
import torch
import torchvision

In [3]:
training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

In [11]:
class MLP(torch.nn.Module):
    def __init__(self, D_in, D_out, act_out, act, layers:list):
        super().__init__()
        #Flatten input to 1D 
        self.layers = [torch.nn.Flatten()]
        #Create input W and b
        self.act_linear(D_in, layers[0], act)
        #Create hidden layer W and b
        for i in range(1,len(layers)):
            self.act_linear(layers[i-1], layers[i], act)
        #Create output W and b
        self.act_linear(layers[-1], D_out, act_out)
        self.layer_stack = torch.nn.Sequential(*self.layers)
    
    def act_linear(self, _in, out, act):
        self.layers += [torch.nn.Linear(_in, out), act()]
        
    def forward(self, x):
        return self.layer_stack(x)        

In [12]:
model = MLP(28*28,10,torch.nn.LeakyReLU,torch.nn.LeakyReLU,[256,128,64])

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:

learning_rate = 1e-3
batch_size = 64
epochs = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.701214  [    0/60000]
loss: 0.663820  [ 6400/60000]
loss: 0.466953  [12800/60000]
loss: 0.537046  [19200/60000]
loss: 0.490364  [25600/60000]
loss: 0.469752  [32000/60000]
loss: 0.388410  [38400/60000]
loss: 0.596348  [44800/60000]
loss: 0.498440  [51200/60000]
loss: 0.514771  [57600/60000]
Test Error: 
 Accuracy: 84.3%, Avg loss: 0.006844 

Epoch 2
-------------------------------
loss: 0.280278  [    0/60000]
loss: 0.373239  [ 6400/60000]
loss: 0.299591  [12800/60000]
loss: 0.392997  [19200/60000]
loss: 0.405260  [25600/60000]
loss: 0.390431  [32000/60000]
loss: 0.327137  [38400/60000]
loss: 0.482946  [44800/60000]
loss: 0.461531  [51200/60000]
loss: 0.424404  [57600/60000]
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.006397 

Epoch 3
-------------------------------
loss: 0.216297  [    0/60000]
loss: 0.345657  [ 6400/60000]
loss: 0.236607  [12800/60000]
loss: 0.320957  [19200/60000]
loss: 0.351551  [25600/60000]
loss: 0.351001  [32000/600